In [4]:
import json
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

data_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\annotated_data_openai.json'
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

texts = data["text"]
labels_raw = data["label_annotated"]
labels = []
for i in labels_raw:
    if i == 0:
        labels.append(1)
    elif i == 1:
        labels.append(0)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, 
    labels, 
    test_size=0.2, 
    random_state=42
)

train_dataset = Dataset.from_dict({
    'text': train_texts,
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'text': val_texts,
    'labels': val_labels
})

raw_datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

In [2]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 120
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 30
    })
})


In [3]:
model_name = "batterydata/batterybert-cased-abstract"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, torch_dtype="auto")

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [6]:
import evaluate

metric = evaluate.load("accuracy")

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer",
                                  eval_strategy="epoch",
                                  num_train_epochs=10)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)


In [10]:
trainer.train()

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.6861875057220459, 'eval_accuracy': 0.7333333333333333, 'eval_runtime': 0.2896, 'eval_samples_per_second': 103.58, 'eval_steps_per_second': 13.811, 'epoch': 1.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.48435792326927185, 'eval_accuracy': 0.8666666666666667, 'eval_runtime': 0.2799, 'eval_samples_per_second': 107.199, 'eval_steps_per_second': 14.293, 'epoch': 2.0}


...

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 1.6543549299240112, 'eval_accuracy': 0.7666666666666667, 'eval_runtime': 0.2571, 'eval_samples_per_second': 116.702, 'eval_steps_per_second': 15.56, 'epoch': 9.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 1.6588170528411865, 'eval_accuracy': 0.7666666666666667, 'eval_runtime': 0.2563, 'eval_samples_per_second': 117.03, 'eval_steps_per_second': 15.604, 'epoch': 10.0}
{'train_runtime': 35.1292, 'train_samples_per_second': 34.16, 'train_steps_per_second': 4.27, 'train_loss': 0.23102294921875, 'epoch': 10.0}


TrainOutput(global_step=150, training_loss=0.23102294921875, metrics={'train_runtime': 35.1292, 'train_samples_per_second': 34.16, 'train_steps_per_second': 4.27, 'total_flos': 315733266432000.0, 'train_loss': 0.23102294921875, 'epoch': 10.0})

In [15]:
trainer.save_model("batterybert-cased-abstract_finetuned_100")

In [5]:
model_name = "batterydata/batterybert-cased-abstract"
tokenizer = AutoTokenizer.from_pretrained(model_name)

from transformers import pipeline
classifier = pipeline("text-classification", model="batterybert-cased-abstract_finetuned_100", tokenizer=tokenizer, device=0)

In [13]:
val_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\annotated_data_openai_1.json'
with open(val_path, 'r', encoding='utf-8') as f:
    val_data = json.load(f)

label_bert = []
score_bert = []
for i in range(len(val_data["text"])):
    val_label = classifier(val_data["text"][i][:512])
    if val_label[0]["label"] == "battery":
        label_bert.append(1)
        score_bert.append(val_label[0]["score"])
    elif val_label[0]["label"] == "non-battery":
        label_bert.append(0)
        score_bert.append(val_label[0]["score"])

val_data["label_bert"] = label_bert
val_data["score_bert"] = score_bert

In [14]:
label_openai = val_data["label_openai"]
label_annotated = val_data["label_annotated"]
print(f"No\tAnno\tOpenAI\tBERT\tBERT Score")

openai_tp = 0
openai_tn = 0
openai_fp = 0
openai_fn = 0
bert_tp = 0
bert_tn = 0
bert_fp = 0
bert_fn = 0

for i in range(len(label_bert)):
    if (label_bert[i] + label_openai[i] + label_annotated[i] != 3) and (label_bert[i] + label_openai[i] + label_annotated[i] != 0):
        print(f"{i}\t{label_annotated[i]}\t{label_openai[i]}\t{label_bert[i]}\t{score_bert[i]}")
        if label_bert[i] == 1 and label_annotated[i] == 0:
            bert_fp += 1
        elif label_bert[i] == 0 and label_annotated[i] == 1:
            bert_fn += 1
        if label_openai[i] == 1 and label_annotated[i] == 0:
            openai_fp += 1
        elif label_openai[i] == 0 and label_annotated[i] == 1:
            openai_fn += 1
    if label_bert[i] == 1 and label_annotated[i] == 1:
        bert_tp += 1
    if label_bert[i] == 0 and label_annotated[i] == 0:
        bert_tn += 1
    if label_openai[i] == 1 and label_annotated[i] == 1:
        openai_tp += 1
    if label_openai[i] == 0 and label_annotated[i] == 0:
        openai_tn += 1
        
print(f"OpenAI True Positive: {openai_tp}")
print(f"OpenAI True Negative: {openai_tn}")
print(f"OpenAI False Positive: {openai_fp}")
print(f"OpenAI False Negative: {openai_fn}")
print(f"Bert True Positive: {bert_tp}")
print(f"Bert True Negative: {bert_tn}")
print(f"Bert False Positive: {bert_fp}")
print(f"Bert False Negative: {bert_fn}")

No	Anno	OpenAI	BERT	BERT Score
7	0	1	0	0.9998339414596558
21	0	1	0	0.9980390667915344
27	1	1	0	0.9996181726455688
34	0	1	0	0.9999748468399048
36	0	1	0	0.9999538660049438
53	0	1	0	0.999754011631012
60	1	1	0	0.9999549388885498
72	1	1	0	0.9999552965164185
79	0	1	0	0.9952448010444641
80	0	1	0	0.9998611211776733
99	0	1	0	0.9999470710754395
OpenAI True Positive: 87
OpenAI True Negative: 5
OpenAI False Positive: 8
OpenAI False Negative: 0
Bert True Positive: 84
Bert True Negative: 13
Bert False Positive: 0
Bert False Negative: 3


In [11]:
val_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\annotated_data_openai.json'
with open(val_path, 'r', encoding='utf-8') as f:
    val_data = json.load(f)

label_bert = []
score_bert = []
for i in range(len(val_data["text"])):
    val_label = classifier(val_data["text"][i][:512])
    if val_label[0]["label"] == "battery":
        label_bert.append(1)
        score_bert.append(val_label[0]["score"])
    elif val_label[0]["label"] == "non-battery":
        label_bert.append(0)
        score_bert.append(val_label[0]["score"])

val_data["label_bert"] = label_bert
val_data["score_bert"] = score_bert

In [12]:
label_openai = val_data["label_openai"]
label_annotated = val_data["label_annotated"]
print(f"No\tAnno\tOpenAI\tBERT\tBERT Score")

openai_tp = 0
openai_tn = 0
openai_fp = 0
openai_fn = 0
bert_tp = 0
bert_tn = 0
bert_fp = 0
bert_fn = 0

for i in range(len(label_bert)):
    if (label_bert[i] + label_openai[i] + label_annotated[i] != 3) and (label_bert[i] + label_openai[i] + label_annotated[i] != 0):
        print(f"{i}\t{label_annotated[i]}\t{label_openai[i]}\t{label_bert[i]}\t{score_bert[i]}")
        if label_bert[i] == 1 and label_annotated[i] == 0:
            bert_fp += 1
        elif label_bert[i] == 0 and label_annotated[i] == 1:
            bert_fn += 1
        if label_openai[i] == 1 and label_annotated[i] == 0:
            openai_fp += 1
        elif label_openai[i] == 0 and label_annotated[i] == 1:
            openai_fn += 1
    if label_bert[i] == 1 and label_annotated[i] == 1:
        bert_tp += 1
    if label_bert[i] == 0 and label_annotated[i] == 0:
        bert_tn += 1
    if label_openai[i] == 1 and label_annotated[i] == 1:
        openai_tp += 1
    if label_openai[i] == 0 and label_annotated[i] == 0:
        openai_tn += 1
        
print(f"OpenAI True Positive: {openai_tp}")
print(f"OpenAI True Negative: {openai_tn}")
print(f"OpenAI False Positive: {openai_fp}")
print(f"OpenAI False Negative: {openai_fn}")
print(f"Bert True Positive: {bert_tp}")
print(f"Bert True Negative: {bert_tn}")
print(f"Bert False Positive: {bert_fp}")
print(f"Bert False Negative: {bert_fn}")

No	Anno	OpenAI	BERT	BERT Score
4	0	1	0	0.9999243021011353
5	0	1	0	0.9952448010444641
8	1	0	1	0.9998788833618164
9	0	0	1	0.9533090591430664
22	1	0	1	0.9998687505722046
25	0	0	1	0.9993022680282593
29	1	1	0	0.9996181726455688
30	0	0	1	0.7364472150802612
36	0	0	1	0.9998989105224609
45	1	1	0	0.9999552965164185
63	0	1	0	0.999754011631012
64	0	0	1	0.9997395873069763
68	1	0	1	0.8610180616378784
95	0	1	0	0.9999748468399048
103	0	1	0	0.9999538660049438
108	0	0	1	0.9999068975448608
112	0	1	0	0.9993190765380859
131	1	1	0	0.9999549388885498
135	0	1	0	0.9999470710754395
136	0	1	0	0.9999133348464966
137	0	1	0	0.9998611211776733
143	0	0	1	0.9995007514953613
OpenAI True Positive: 84
OpenAI True Negative: 54
OpenAI False Positive: 9
OpenAI False Negative: 3
Bert True Positive: 84
Bert True Negative: 56
Bert False Positive: 7
Bert False Negative: 3
